In [7]:
import pandas as pd
import math

In [8]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 10)

## Load Data

In [9]:
clean_data = pd.read_csv('./data/farmersmarkets.csv', low_memory=False)
last_season = pd.read_csv('./data/seasons.csv')

In [10]:
database = clean_data.merge(last_season, how='left', on='FMID')
database

,FMID,MarketName,Website,Facebook,Twitter,Youtube,OtherMedia,Street,City,County,State,zip,Season1Date 1,Season1Time 1,Season1Time 2,Season1Time 3,Season1Time 4,Season1Time 5,Season1Time 6,Season1Time 7,Season1Time 8,Season2Date 1,Season2Time 1,Season2Time 2,Season2Time 3,Season2Time 4,Season2Time 5,Season2Time 6,Season2Time 7,Season2Time 8,Season3Date 1,Season3Time 1,Season3Time 2,Season3Time 3,Season3Time 4,Season3Time 5,Season3Time 6,Season3Time 7,Season4Date 1,Season4Time 1,Season4Time 2,Season4Time 3,x,y,Location,Credit,WIC,WICcash,SFMNP,SNAP,Organic,Bakedgoods,Cheese,Crafts,Flowers,Eggs,Seafood,Herbs,Vegetables,Honey,Jams,Maple,Meat,Nursery,Nuts,Plants,Poultry,Prepared,Soap,Trees,Wine,Coffee,Beans,Fruits,Grains,Juices,Mushrooms,PetFood,Tofu,WildHarvested,updateTime,MostRecentOpeningDate
0,1012063,Caledonia Farmers Market Association - Danville,https://sites.google.com/site/caledoniafarmers...,https://www.facebook.com/Danville.VT.Farmers.M...,NaN,NaN,NaN,NaN,Danville,Caledonia,Vermont,05828,06/08/2016 to 10/12/2016,Wed: 9:00 AM-1:00 PM,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-72.140305,44.411013,NaN,Y,Y,N,Y,N,Y,Y,Y,Y,Y,Y,N,Y,Y,Y,Y,Y,Y,N,N,Y,Y,Y,Y,Y,N,Y,Y,Y,N,Y,N,Y,N,N,2018-04-06T12:10:09Z,2016-10-12
1,1011871,Stearns Homestead Farmers' Market,http://Stearnshomestead.com,NaN,NaN,NaN,NaN,6975 Ridge Road,Parma,Cuyahoga,Ohio,44130,06/25/2016 to 10/01/2016,Sat: 9:00 AM-1:00 PM,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-81.728597,41.375118,NaN,Y,Y,N,Y,Y,-,Y,N,N,Y,Y,N,Y,Y,Y,Y,Y,Y,N,N,Y,N,N,N,N,N,N,N,Y,N,N,N,Y,N,N,2016-09-04T08:05:17Z,2016-10-01
2,1011878,100 Mile Market,http://www.pfcmarkets.com,https://www.facebook.com/100MileMarket/?fref=ts,NaN,NaN,https://www.instagram.com/100milemarket/,507 Harrison St,Kalamazoo,Kalamazoo,Michigan,49007,05/04/2016 to 10/12/2016,Wed: 3:00 PM-7:00 PM,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-85.574887,42.296024,NaN,Y,Y,N,Y,Y,N,Y,Y,N,Y,Y,N,Y,Y,Y,Y,Y,Y,N,N,N,Y,Y,Y,N,N,N,N,Y,N,N,N,N,N,N,2017-03-07T07:20:33Z,2016-10-12
3,1009364,106 S. Main Street Farmers Market,http://thetownofsixmile.wordpress.com/,NaN,NaN,NaN,NaN,106 S. Main Street,Six Mile,NaN,South Carolina,29682,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-82.818700,34.804200,NaN,Y,N,N,N,N,-,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2013-01-01T00:00:00Z,NaN
4,1010691,10th Steet Community Farmers Market,NaN,NaN,NaN,NaN,http://agrimissouri.com/mo-grown/grodetail.php...,10th Street and Poplar,Lamar,Barton,Missouri,64759,04/02/2014 to 11/30/2014,Wed: 3:00 PM-6:00 PM,Sat: 8:00 AM-1:00 PM,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-94.274619,37.495628,NaN,Y,N,N,N,N,-,Y,N,Y,N,Y,N,Y,Y,Y,Y,N,Y,N,N,Y,Y,Y,Y,N,N,N,N,Y,N,N,N,N,N,N,2016-04-10T09:49:46Z,2014-11-30
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8660,1004767,Zia Bernalillo Farmers' Market,http://www.eatfreshnm.org/,https://www.facebook.com/farmersmarketsnm?ref=hl,NaN,NaN,NaN,335 S. Camino del Pueblo,Bernalillo,Sandoval,New Mexico,NaN,07/04/2014 to 10/24/2014,Fri: 4:00 PM-7:00 PM,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-106.546840,35.313704,"Faith-based institution (e.g., church, mosque,...",N,Y,N,Y,Y,-,Y,Y,Y,N,Y,N,N,Y,Y,N,N,Y,N,N,N,N,N,N,N,N,N,Y,Y,N,N,N,N,N,N,2014-05-06T02:40:25Z,2014-10-24
8661,1000778,Zimmerman Farmers M

## Build Recommendation System

In [175]:
def haversine_distance(lat1, lon1, lat2, lon2):
    # Convert latitude and longitude from degrees to radians
    lat1_rad = math.radians(lat1)
    lon1_rad = math.radians(lon1)
    lat2_rad = math.radians(lat2)
    lon2_rad = math.radians(lon2)

    # Haversine formula
    dlat = lat2_rad - lat1_rad
    dlon = lon2_rad - lon1_rad
    a = math.sin(dlat / 2) ** 2 + math.cos(lat1_rad) * math.cos(lat2_rad) * math.sin(dlon / 2) ** 2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))

    # Radius of the Earth in kilometers
    R = 6371.0

    # Calculate the distance
    distance = R * c

    return distance

In [5]:
def get_recommended_farmers_markets(
        database, county=None, gps_longitude=None, gps_latitude=None, preferred_payment=None, items=[]
    ):
    if not county:
        print("Please provide the country you live in.")
        
    elif not items:
        print("Please provide items you need to buy.")

    else:
        # filter by county
        county = county.title()
        eligible_farmers_markets = database.loc[(database.County == county), ]

        # filter by payment
        if not preferred_payment:
            print("Please provide preferred payment method.")
        else:
            if preferred_payment not in ['Credit', 'WIC', 'WICcash', 'SFMNP', 'SNAP']:
                print(
                    "Payment not accepted. Accepted payment include: 'Credit', 'WIC', 'WICcash', 'SFMNP', 'SNAP'."
                )
            else:
                eligible_farmers_markets = eligible_farmers_markets.loc[
                    eligible_farmers_markets[preferred_payment] == 'Y',
                ]
                
                # filter by products
                if len(items) > 0:
                    for i in items:
                        eligible_farmers_markets = eligible_farmers_markets.loc[
                            eligible_farmers_markets[i] == 'Y',
                        ]
                
                # rank order based on distance
                if not gps_longitude or not gps_latitude:
                    print(
                        '''
                        Please note that the recommendations are not ranked ordered. 
                        If you would like to have them ranked based on distance, 
                        please provide your GBS locations.
                        '''
                    )
                    
                    recommendations = eligible_farmers_markets.reset_index(drop=True)
                else:
                    eligible_farmers_markets['distance'] = [
                        haversine_distance(gps_latitude, gps_longitude, y, x) 
                        for x, y in zip(eligible_farmers_markets['x'], eligible_farmers_markets['y'])
                    ]

                    recommendations = eligible_farmers_markets.sort_values(['distance']).reset_index(drop=True)

                recommendations = recommendations[[
                    'FMID', 'MarketName', 'Website', 'Facebook', 'Twitter', 'Youtube',
                    'OtherMedia', 'Street', 'City', 'County', 'State', 'zip', 'Location', 'MostRecentOpeningDate'
                ]]
                return recommendations
    

## Test Use Cases

In [184]:
get_recommended_farmers_markets(database)

Please provide the country you live in.


In [185]:
get_recommended_farmers_markets(database, county='Sandoval')

Please provide items you need to buy.


In [186]:
get_recommended_farmers_markets(database, county='Sandoval', items=['Vegetables'])

Please provide preferred payment method.


In [187]:
get_recommended_farmers_markets(database, county='Sandoval', items=['Vegetables'], preferred_payment="cash")

Payment not accepted. Accepted payment include: 'Credit', 'WIC', 'WICcash', 'SFMNP', 'SNAP'.


In [211]:
get_recommended_farmers_markets(database, county='Sandoval', items=['Vegetables'], preferred_payment="WICcash")

Please note that the recommendations are not ranked ordered. If you would like to have them rank ordered based on distance, please provide your GBS locations.


,FMID,MarketName,Website,Facebook,Twitter,Youtube,OtherMedia,Street,City,County,State,zip,Location,MostRecentOpeningDate
0,1006861,"Cuba, New Mexico Farmers Market",NaN,NaN,NaN,NaN,NaN,NM 126 St. Francis of Assisi Park,Cuba,Sandoval,New Mexico,87013,Federal/State government building grounds,NaT


In [11]:
get_recommended_farmers_markets(database, county='los angeles'.capitalize(), items=['Vegetables'], preferred_payment="WICcash")


                        Please note that the recommendations are not ranked ordered. 
                        If you would like to have them ranked based on distance, 
                        please provide your GBS locations.
                        


,FMID,MarketName,Website,Facebook,Twitter,Youtube,OtherMedia,Street,City,County,State,zip,Location,MostRecentOpeningDate
0,1010003,Atwater Village Farmers' Market,http://www.seela.org/atwater-village-farmers-m...,https://www.facebook.com/pages/Atwater-Village...,https://twitter.com/AtwaterFM,NaN,www.instagram.com/atwatervillagefm,3250 Glendale Blvd,Los Angeles,Los Angeles,California,90039,Private business parking lot,2014-12-31
1,1005208,Central Ave Farmers Market,http://www.farmernet.com,NaN,NaN,NaN,NaN,4301 Central Ave at 43rd st,Los angeles California,Los Angeles,California,90011,Local government building grounds,NaN
2,1010147,Cerritos Farmers Market,http://www.goodveg.org,https://www.facebook.com/harborareafarmersmarkets,NaN,NaN,NaN,17870 Park plaza Drive,Cerritos,Los Angeles,California,90703,Local government building grounds,NaN
3,1002993,East Los Angeles Farmers Market,http://www.velasfarmersmarkets.org,https://www.facebook.com/pages/East-Los-Angele...,https://twitter.com/Vela_ELA?refsrc=email,NaN,NaN,4801 East 3rd Street,Los Angeles,Los Angeles,California,90022,NaN,2013-12-31
4,1008271,East Los Angeles WEDNESDAY Farmers Market,http://www.velasfarmersmarkets.org,NaN,NaN,NaN,NaN,4848 East Civic Center Way,East Los Angeles,Los Angeles,California,90022,NaN,2015-09-30
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17,1012334,Santa Monica Farmers Market - Saturday Downtown,http://www.smgov.net/portals/farmersmarket,Santa Monica Farmers Market,@smfms,NaN,Instagram @smfms,2nd Street & Arizona Avenue,Santa Monica,Los Angeles,California,90405,NaN,2016-12-31
18,1003707,South Gate Farmers Market,http://www.goodveg.org,https://www.facebook.com/harborareafarmersmarkets,NaN,NaN,NaN,4855 Tweedy Blvd,South Gate,Los Angeles,California,90280,Local government building grounds,2014-12-31
19,1011757,The WALL Food + Flowers + Farmers Market,http://www.communityevent.org,The WALL Food Flowers,The WALL,NaN,TheWallLA on Instagram,755 Wall Street,Los Angeles,Los Angeles,California,90014,NaN,2016-12-31
20,1010037,Watts Healthy Farmers' Market,http://www.seela.org/,https://www.facebook.com/WattsHealthyFarmersMa...,NaN,NaN,NaN,103rd Street and Central,Los Angeles,Los Angeles,California,90002,Local government building grounds,2014-12-31


In [228]:
get_recommended_farmers_markets(database, county='los angeles'.capitalize(), items=['Vegetables'], preferred_payment="WICcash", gps_latitude=34.12638, gps_longitude=-118.212365)

,FMID,MarketName,Website,Facebook,Twitter,Youtube,OtherMedia,Street,City,County,State,zip,Location,MostRecentOpeningDate
0,1010003,Atwater Village Farmers' Market,http://www.seela.org/atwater-village-farmers-m...,https://www.facebook.com/pages/Atwater-Village...,https://twitter.com/AtwaterFM,NaN,www.instagram.com/atwatervillagefm,3250 Glendale Blvd,Los Angeles,Los Angeles,California,90039,Private business parking lot,2014-12-31
1,1010019,Echo Park Farmers' Market,http://seela.org,https://www.facebook.com/pages/Echo-Park-Farme...,NaN,NaN,NaN,Logan St. and Sunset Blvd,Los Angeles,Los Angeles,California,90026,NaN,2015-12-31
2,1011757,The WALL Food + Flowers + Farmers Market,http://www.communityevent.org,The WALL Food Flowers,The WALL,NaN,TheWallLA on Instagram,755 Wall Street,Los Angeles,Los Angeles,California,90014,NaN,2016-12-31
3,1008271,East Los Angeles WEDNESDAY Farmers Market,http://www.velasfarmersmarkets.org,NaN,NaN,NaN,NaN,4848 East Civic Center Way,East Los Angeles,Los Angeles,California,90022,NaN,2015-09-30
4,1002993,East Los Angeles Farmers Market,http://www.velasfarmersmarkets.org,https://www.facebook.com/pages/East-Los-Angele...,https://twitter.com/Vela_ELA?refsrc=email,NaN,NaN,4801 East 3rd Street,Los Angeles,Los Angeles,California,90022,NaN,2013-12-31
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17,1003706,Long Beach Uptown Farmers Market,http://www.goodveg.org,https://www.facebook.com/harborareafarmersmarkets,NaN,NaN,NaN,4600 Atlantic Avenue,Long Beach,Los Angeles,California,90807,Closed-off public street,2014-12-31
18,1010147,Cerritos Farmers Market,http://www.goodveg.org,https://www.facebook.com/harborareafarmersmarkets,NaN,NaN,NaN,17870 Park plaza Drive,Cerritos,Los Angeles,California,90703,Local government building grounds,NaT
19,1003673,Kaiser South Bay Farmers' Market,http://www.lacienegafarmersmarket.com,https://www.facebook.com/kaisersouthbayfarmers...,https://twitter.com/MNPFarmersMkt,NaN,NaN,25825 S. Vermont Avenue,Harbor City,Los Angeles,California,90710,Healthcare Institution,2014-12-01
20,1010146,Long Beach Downtown Farmers Market,http://www.goodveg.org,https://www.facebook.com/harborareafarmersmarkets,NaN,NaN,NaN,212 West Third Street,Long Beach,Los Angeles,California,90802,Local government building grounds,2014-12-31
